In [6]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import GPT2LMHeadModel

In [7]:
# 스페셜 토큰 정의
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = "</s>"
EOS = "</s>"
SENT = '<unused1>'
PAD = "<pad>"
MASK = "<unused0>"
# 사전학습된 토크나이저
TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token=BOS, eos_token=EOS, unk_token="<unk>", pad_token=PAD, mask_token=MASK)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [8]:
max_input_length = TOKENIZER.model_max_length  # 모델의 최대 입력 길이
vocab_size = TOKENIZER.vocab_size  # 토크나이저의 어휘 크기
print(f"max_input_length: {max_input_length}, vocab_size: {vocab_size}")

max_input_length: 1000000000000000019884624838656, vocab_size: 51200


In [9]:
model_state_dict = torch.load('./model/chatbot_model2.pth')
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [11]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(TOKENIZER.encode(Q_TKN + q + SENT + A_TKN + a)).unsqueeze(dim=0)
            model = model.to("cpu")
            pred = model(input_ids)
            pred = pred.logits
            gen = TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))

IndexError: index out of range in self